
Using: https://platform.here.com/services/details/hrn:here:service::olp-here:matrix-routing-8/overview

To get travel time matrix

Base url is: https://matrix.router.hereapi.com/v8

## Part 1: Set up Libs and API Base URL


In [1]:
import requests
import pandas as pd
import json

Set up the Base URL and read in the API key:

In [2]:
API_Key = open( r'/mnt/d/OneDrive - University College London/PhD/UCL/UCL-PhD-RP/M.Res/7_API_Keys/here.txt', 'r').readline().strip()

BASE_URL = 'https://matrix.router.hereapi.com/v8'

Read the msoa all lat lon coords:

In [3]:
msoa_london = pd.read_csv(r'/mnt/d/University College London/Chen, Huanfa - Dissertation_Mengyu_Ding/Data/msoa/processed/gld_msoa_centroids.csv', encoding='latin1')

take a look at the msoa data:

In [4]:
msoa_london.head()

,hclnm,hclnmw,label,msoa21cd,msoa21nm,name,Boroughs,centroid,lat,lon
0,Plumstead High Street,NaN,E92000001E12000007E09000011E02000318,E02000318,Greenwich 006,Greenwich 006,Greenwich,POINT (0.0956181263047325 51.48826506306727),51.488265,0.095618
1,Fulham Broadway,NaN,E92000001E12000007E09000013E02000388,E02000388,Hammersmith and Fulham 017,Hammersmith and Fulham 017,Hammersmith and Fulham,POINT (-0.1984452219606496 51.48315729022629),51.483157,-0.198445
2,Streatham Vale,NaN,E92000001E12000007E09000022E02000652,E02000652,Lambeth 035,Lambeth 035,Lambeth,POINT (-0.1380462958677475 51.41504113246437),51.415041,-0.138046
3,Queen's Park Gardens,NaN,E92000001E12000007E09000033E02000963,E02000963,Westminster 004,Westminster 004,Westminster,POINT (-0.209370884304258 51.52907905830605),51.529079,-0.209371
4,Streatham Common,NaN,E92000001E12000007E09000022E02000651,E02000651,Lambeth 034,Lambeth 034,Lambeth,POINT (-0.1259419896669564 51.4191529314827),51.419153,-0.125942


see if the msoa21cd column is unique:

In [5]:
# see if the column msoa21cd is unique:
msoa_london['msoa21cd'].nunique() == msoa_london.shape[0]

True

We can use this as the msoa id then.
Also the OD matrix origin will be set to rows:

| Origin/Destination | Destination 1 | Destination 2 | Destination 3 |
|--------------------|---------------|---------------|---------------|
| Origin 1           | 10            | 5             | 3             |
| Origin 2           | 2             | 8             | 6             |
| Origin 3           | 4             | 7             | 1             |




__Departure time__

23rd. July 2024. 7:30 am. As the baseline.

Getting ready to send out the big query.
run a 1002 x 1003
then a 1 x 1003, and piece together after running the big request. -- maybe not. it looks ok to run 1003x1003 asynch

In [6]:
# follow this format: [{"lat": ..., "lng": ...}, ...]

# new dict origin: 1002 maoas
origin_coord_dict = {}
origin_coord = []

# new dict destination:
dest_coord = []
dest_coord_dict = {}

read in the dataframe and the list is ready to be fed to API payload:

In [9]:
msoa_london.iloc[1, 0]

'Fulham Broadway'

In [10]:
# for reference dict, read the msoa_london, use msoa21cd as key, lat and lon as value:
for i in range(msoa_london.shape[0]):
    msoaID = msoa_london.iloc[i, 3]
    lat = msoa_london.iloc[i, 8]
    lon = msoa_london.iloc[i, 9]
    origin_coord.append({"lat": lat, "lng": lon})
    origin_coord_dict[msoaID] = {"lat": lat, "lng": lon}
    dest_coord.append({"lat": lat, "lng": lon})
    dest_coord_dict[msoaID] = {"lat": lat, "lng": lon}

In [11]:
dest_coord 

[{'lat': 51.48826506306727, 'lng': 0.0956181263047325},
 {'lat': 51.48315729022629, 'lng': -0.1984452219606495},
 {'lat': 51.41504113246437, 'lng': -0.1380462958677474},
 {'lat': 51.52907905830605, 'lng': -0.209370884304258},
 {'lat': 51.4191529314827, 'lng': -0.1259419896669564},
 {'lat': 51.36949206068002, 'lng': -0.077636700171988},
 {'lat': 51.51620181303971, 'lng': 0.0774631221982},
 {'lat': 51.56360242260222, 'lng': -0.2344250853261522},
 {'lat': 51.474611437273566, 'lng': -0.0673270348400329},
 {'lat': 51.466601391774674, 'lng': -0.1124073736023934},
 {'lat': 51.4984182398741, 'lng': -0.2305090479379697},
 {'lat': 51.53792786833988, 'lng': -0.0936864361978526},
 {'lat': 51.49332975613745, 'lng': -0.1691014538105725},
 {'lat': 51.572461188240176, 'lng': -0.10859264639662},
 {'lat': 51.370818324305525, 'lng': -0.3038262306837527},
 {'lat': 51.54433564276359, 'lng': -0.131335891288934},
 {'lat': 51.43083272052458, 'lng': -0.1668618849503236},
 {'lat': 51.570889960285584, 'lng': -0.

Now, initialize an empty distance matrix to store the result:

In [7]:
# initialize a pandas dataframe to store the distance matrix, row index is msoa21cd, column index is also msoa21cd:
distance_matrix = pd.DataFrame(index=msoa_london['msoa21cd'], columns=msoa_london['msoa21cd'])

In [8]:
display(distance_matrix)

msoa21cd,E02000318,E02000388,E02000652,E02000963,E02000651,E02000216,E02000746,E02000103,E02000828,E02000628,...,E02000259,E02000817,E02000879,E02000865,E02000173,E02000481,E02000885,E02000674,E02000975,E02000211
msoa21cd,,,,,,,,,,,,,,,,,,,,,
E02000318,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
E02000388,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
E02000652,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
E02000963,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
E02000651,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
E02000481,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
E02000885,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
E02000674,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Now the TTM, travel time matrix:

In [12]:
travel_time_mat = pd.DataFrame(index=msoa_london['msoa21cd'], columns=msoa_london['msoa21cd'])

In [37]:
origin_coord

[{'lat': 51.48826506306727, 'lng': 0.0956181263047325},
 {'lat': 51.48315729022629, 'lng': -0.1984452219606495},
 {'lat': 51.41504113246437, 'lng': -0.1380462958677474},
 {'lat': 51.52907905830605, 'lng': -0.209370884304258},
 {'lat': 51.4191529314827, 'lng': -0.1259419896669564},
 {'lat': 51.36949206068002, 'lng': -0.077636700171988},
 {'lat': 51.51620181303971, 'lng': 0.0774631221982},
 {'lat': 51.56360242260222, 'lng': -0.2344250853261522},
 {'lat': 51.474611437273566, 'lng': -0.0673270348400329},
 {'lat': 51.466601391774674, 'lng': -0.1124073736023934},
 {'lat': 51.4984182398741, 'lng': -0.2305090479379697},
 {'lat': 51.53792786833988, 'lng': -0.0936864361978526},
 {'lat': 51.49332975613745, 'lng': -0.1691014538105725},
 {'lat': 51.572461188240176, 'lng': -0.10859264639662},
 {'lat': 51.370818324305525, 'lng': -0.3038262306837527},
 {'lat': 51.54433564276359, 'lng': -0.131335891288934},
 {'lat': 51.43083272052458, 'lng': -0.1668618849503236},
 {'lat': 51.570889960285584, 'lng': -0.

## Part 2 Getting Ready for the API Calls

define payload:
centre of Greater London:
https://londonist.com/2014/04/where-is-the-centre-of-london-an-update

In [49]:
import os
from here_location_services import LS
from here_location_services.config.matrix_routing_config import (
BoundingBoxRegion,
AutoCircleRegion,
MATRIX_ATTRIBUTES,
WorldRegion,
)

#LS_API_KEY = os.environ.get("LS_API_KEY") # Get API KEY from environment.
ls = LS(api_key=API_Key) # Create Location Services object using API KEY.
origins = origin_coord
destinations = dest_coord # List of destination coordinates.
region_definition = AutoCircleRegion()
matrix_attributes = [MATRIX_ATTRIBUTES.distances]
result = ls.matrix(
origins=origins,
region_definition=region_definition,
matrix_attributes=matrix_attributes,
async_req=True
)
result.response

{'matrixId': 'ffff9c21-ebbf-443b-a1e9-0a8ffb914949',
 'matrix': {'numOrigins': 1002,
  'numDestinations': 1002,
  'distances': [0,
   25005,
   23852,
   29857,
   23176,
   22664,
   18127,
   46646,
   13627,
   17166,
   28219,
   20323,
   22510,
   22641,
   76038,
   22171,
   25071,
   58301,
   10868,
   2307,
   46609,
   46756,
   100375,
   35451,
   53805,
   15873,
   64567,
   39693,
   28679,
   10076,
   34101,
   23285,
   22668,
   24146,
   22770,
   20299,
   36061,
   59394,
   37533,
   53259,
   27173,
   47783,
   36729,
   36969,
   100493,
   21462,
   66307,
   24480,
   35506,
   20000,
   22709,
   27820,
   22205,
   16975,
   1942,
   27879,
   50361,
   4420,
   77873,
   49130,
   18886,
   20798,
   20885,
   60955,
   23510,
   49004,
   15776,
   25278,
   36051,
   19970,
   23147,
   31511,
   49502,
   39568,
   103207,
   50032,
   15300,
   16112,
   28840,
   19376,
   46963,
   13549,
   102255,
   14514,
   26762,
   50881,
   29258,
   17454

In [52]:
result.response['matrix']

{'numOrigins': 1002,
 'numDestinations': 1002,
 'distances': [0,
  25005,
  23852,
  29857,
  23176,
  22664,
  18127,
  46646,
  13627,
  17166,
  28219,
  20323,
  22510,
  22641,
  76038,
  22171,
  25071,
  58301,
  10868,
  2307,
  46609,
  46756,
  100375,
  35451,
  53805,
  15873,
  64567,
  39693,
  28679,
  10076,
  34101,
  23285,
  22668,
  24146,
  22770,
  20299,
  36061,
  59394,
  37533,
  53259,
  27173,
  47783,
  36729,
  36969,
  100493,
  21462,
  66307,
  24480,
  35506,
  20000,
  22709,
  27820,
  22205,
  16975,
  1942,
  27879,
  50361,
  4420,
  77873,
  49130,
  18886,
  20798,
  20885,
  60955,
  23510,
  49004,
  15776,
  25278,
  36051,
  19970,
  23147,
  31511,
  49502,
  39568,
  103207,
  50032,
  15300,
  16112,
  28840,
  19376,
  46963,
  13549,
  102255,
  14514,
  26762,
  50881,
  29258,
  17454,
  23981,
  40310,
  15214,
  22159,
  28323,
  21536,
  21142,
  21239,
  30618,
  39208,
  54401,
  22932,
  50440,
  52145,
  27546,
  43818,
  21819

In [46]:
matrix_result = result.response['matrix']


In [48]:
matrix_result['distances']

[0,
 24464,
 22933,
 27238,
 22257,
 22664,
 18127,
 46646,
 13627,
 17224,
 27909,
 20284,
 22471,
 22499,
 78696,
 22171,
 24822,
 58742,
 10868,
 1891,
 46557,
 47925,
 103033,
 35451,
 53805,
 16440,
 64567,
 39693,
 28679,
 10076,
 36380,
 24999,
 22668,
 24141,
 24252,
 20299,
 110063,
 59192,
 110527,
 53966,
 26801,
 47783,
 36085,
 36969,
 100493,
 21450,
 110381,
 23939,
 34862,
 20762,
 22709,
 27820,
 22205,
 16975,
 1942,
 27879,
 50361,
 4004,
 80531,
 49130,
 18886,
 21072,
 20885,
 63791,
 23510,
 49004,
 15776,
 28250,
 36051,
 19970,
 23381,
 30504,
 49502,
 107969,
 105865,
 50032,
 15300,
 16112,
 27089,
 20639,
 46963,
 13901,
 65076,
 14514,
 27985,
 50881,
 28339,
 16260,
 23981,
 49152,
 14937,
 21240,
 27951,
 21536,
 21142,
 21239,
 34548,
 39208,
 54401,
 22391,
 50440,
 52145,
 47343,
 43818,
 21926,
 35999,
 30218,
 17411,
 6837,
 30091,
 4065,
 23698,
 14473,
 53183,
 35988,
 18548,
 22688,
 18295,
 60963,
 12537,
 28937,
 25006,
 74859,
 38537,
 22958,
 5

In [50]:
import csv

distances = matrix_result['distances']
distance_matrix = [
    distances[i * len(destinations):(i + 1) * len(destinations)]
    for i in range(len(origins))
]

# Save to CSV
with open('routing_matrix.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Origin", "Destination", "Distance"])
    for origin_idx, origin in enumerate(origins):
        for dest_idx, destination in enumerate(destinations):
            distance = distance_matrix[origin_idx][dest_idx]
            writer.writerow([origin, destination, distance])

print("Results saved to routing_matrix.csv")

Results saved to routing_matrix.csv


In [54]:
#number of unique values in distance_matrix:
len(set([item for sublist in distance_matrix for item in sublist]))

94059

In [63]:
len(matrix_result['distances'])

1004004

In [69]:
#LS_API_KEY = os.environ.get("LS_API_KEY") # Get API KEY from environment.
ls = LS(api_key=API_Key) # Create Location Services object using API KEY.
origins = origin_coord
destinations = dest_coord # List of destination coordinates.
region_definition = AutoCircleRegion()
matrix_attributes = [MATRIX_ATTRIBUTES.distances, MATRIX_ATTRIBUTES.travelTimes]
departure_time = "2024-07-23T07:30:00+01:00"
result = ls.matrix(
origins=origins,
region_definition=region_definition,
matrix_attributes=matrix_attributes,
async_req=True
)
result.response

{'matrixId': '09663204-83d5-46e0-a370-8879e17f5124',
 'matrix': {'numOrigins': 1002,
  'numDestinations': 1002,
  'travelTimes': [0,
   4477,
   3629,
   4762,
   3493,
   3309,
   2438,
   4492,
   2234,
   2940,
   5155,
   3041,
   3940,
   3607,
   4795,
   3460,
   3956,
   5902,
   1684,
   308,
   4488,
   4685,
   5796,
   3490,
   5462,
   1936,
   5842,
   3912,
   2966,
   1453,
   5491,
   3646,
   3470,
   3148,
   3343,
   2924,
   5762,
   5918,
   5885,
   5030,
   4382,
   4419,
   5141,
   3712,
   5705,
   2850,
   5992,
   4293,
   4762,
   3513,
   3247,
   3104,
   2518,
   2460,
   363,
   4555,
   4842,
   605,
   5065,
   4668,
   2763,
   3209,
   2444,
   6032,
   3666,
   4897,
   2303,
   4593,
   3571,
   3078,
   4219,
   4382,
   4756,
   6066,
   5792,
   4657,
   2178,
   2596,
   3220,
   3349,
   4519,
   2168,
   5841,
   2184,
   4942,
   4663,
   4695,
   2808,
   2728,
   3809,
   2490,
   3408,
   4562,
   3669,
   3249,
   3291,
   4670,
   387

In [72]:
matrix_result = result.response['matrix']

In [73]:
distances = matrix_result['distances']
distance_matrix = [
    distances[i * len(destinations):(i + 1) * len(destinations)]
    for i in range(len(origins))
]

# Save to CSV
with open(r'/mnt/d/University College London/Chen, Huanfa - Dissertation_Mengyu_Ding/Data/msoa/raw/routing_matrix_distance.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Origin", "Destination", "Distance"])
    for origin_idx, origin in enumerate(origins):
        for dest_idx, destination in enumerate(destinations):
            distance = distance_matrix[origin_idx][dest_idx]
            writer.writerow([origin, destination, distance])

In [75]:
#number of unique values in distance_matrix:
len(set([item for sublist in distance_matrix for item in sublist]))

90170

In [76]:
len(matrix_result['distances'])

1004004

In [74]:
travelTimes = matrix_result['travelTimes']
travelTimes_matrix = [
    travelTimes[i * len(destinations):(i + 1) * len(destinations)]
    for i in range(len(origins))
]

# Save to CSV
with open(r'/mnt/d/University College London/Chen, Huanfa - Dissertation_Mengyu_Ding/Data/msoa/here/routing_matrix_time.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Origin", "Destination", "time"])
    for origin_idx, origin in enumerate(origins):
        for dest_idx, destination in enumerate(destinations):
            travelTime = travelTimes_matrix[origin_idx][dest_idx]
            writer.writerow([origin, destination, travelTime])

In [77]:
len(set([item for sublist in travelTimes_matrix for item in sublist]))

6727

In [78]:
len(matrix_result['distances'])

1004004

In [40]:
req_body = {
    "origins": origin_coord,
    #"destinations": [...],  // if omitted, origins are used as destinations
    "departureTime": "2024-07-23T07:30:00-01:00",
    "regionDefinition": {
        "type": "circle",
        "center": {"lat": 51.510244, "lng": -0.12571990}, # London Centre defined in the link above
        "radius": 200000 # radius 100km
    },
    "matrixAttributes": ["distances"]
}
headers = {'Content-Type': 'application/json'}

https://www.here.com/docs/bundle/matrix-routing-api-api-reference/page/index.html#tag/Matrix-Calculation


In [41]:
req_body_mode2 = {
    "origins": origin_coord,
    #"destinations": [...],  // if omitted, origins are used as destinations
    "departureTime": "2024-07-23T07:30:00-01:00",
    "regionDefinition": {
        "type": "world"
    },
    "matrixAttributes": ["distances"]
}
headers = {'Content-Type': 'application/json'}

In [42]:
# now sed the request to the API:
#url = f'{BASE_URL}/matrix&apiKey={API_Key}' # the url to send the request to
url = f'{BASE_URL}/matrix?apiKey={API_Key}&async=true'
#response = requests.post(url, headers=headers, data=json.dumps(req_body)) # send the request
response = requests.post(url,data=json.dumps(req_body)) # send the request
# prints the response code:
print(response.status_code)
response_json = response.json() # parse the response as JSON
# save the response to a file
with open(r'/mnt/d/University College London/Chen, Huanfa - Dissertation_Mengyu_Ding/Data/msoa/here/here_distance_matrix.json', 'w') as f:
    json.dump(response.json(), f)

202


In [20]:
response_json

{'matrixId': 'cf97a9a2-c1f5-4e17-8a2c-534d4a7bcfb3',
 'status': 'accepted',
 'statusUrl': 'https://aws-eu-west-1.matrix.router.hereapi.com/v8/matrix/cf97a9a2-c1f5-4e17-8a2c-534d4a7bcfb3/status'}

In [13]:
# Define the request payload
payload = {
    "async": True,
    "region": "your_region_here",
    "matrixSize": {
        "rows": 1004,
        "columns": 1004
    },
    # Add other necessary parameters from the spec if needed
    "origins": origin_coord  # Replace with your actual origin coordinates
    #"destinations": dest_coord ,  / if omitted same as origins # Replace with your actual destination coordinates
}

In [ ]:
payload

In [ ]:
# Define headers, including the API key if needed
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {API_Key}"
}

# Make the asynchronous request
response = requests.post(BASE_URL, json=payload, headers=headers)

# Check the response
if response.status_code == 202:
    print("Request accepted. Check the status of your request using the provided URL.")
    status_url = response.json().get("statusUrl")
    print(f"Status URL: {status_url}")
else:
    print(f"Request failed with status code: {response.status_code}")
    print(f"Response: {response.text}")

In [ ]:
# save the response to a json file:
with open(r'/mnt/d/University College London/Chen, Huanfa - Dissertation_Mengyu_Ding/Data/msoa/raw/here_distance_matrix.json', 'w') as f:
    json.dump(response.json(), f)

In [ ]:
# save the response to the distance matrix:
for i in range(1004):
    for j in range(1004):
        distance_matrix.iloc[i, j] = response.json()['matrix'][i][j]['summary']['distance']

In [ ]:
# Example function to split the matrix into smaller chunks
def split_matrix(coordinates, max_size):
    """Splits the list of coordinates into smaller chunks."""
    for i in range(0, len(coordinates), max_size):
        for j in range(0, len(coordinates), max_size):
            yield coordinates[i:i + max_size], coordinates[j:j + max_size]

# Function to calculate travel time matrix
def calculate_travel_time_matrix(api_key, origins, destinations):
    headers = {
        'Content-Type': 'application/json'
    }
    params = {
        'apiKey': api_key
    }
    body = {
        'origins': [{'lat': coord[0], 'lng': coord[1]} for coord in origins],
        'destinations': [{'lat': coord[0], 'lng': coord[1]} for coord in destinations],
        'regionDefinition': {
            'type': 'circle',
            'center': {'lat': origins[0][0], 'lng': origins[0][1]},
            'radius': 10000
        }
    }
    response = requests.post(BASE_URL, headers=headers, params=params, data=json.dumps(body))
    if response.status_code == 200:
        return response.json()
    else:
        response.raise_for_status()

# Example usage
coordinates = [(lat, lon) for lat, lon in zip(range(4835), range(4835))]  # Replace with actual coordinates
max_matrix_size = 100  # Example, verify the actual limit from the HERE API documentation

for origins, destinations in split_matrix(coordinates, max_matrix_size):
    try:
        result = calculate_travel_time_matrix(API_KEY, origins, destinations)
        print(result)
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")



Storing the matrix:

1. File Naming: Consistent naming and file dir storage for the MSOA matrix

2. Any thoughts in storing it in a NoSQL database? Or maybe GraphDB? -- Simple CSV is fine.

## Part 3 Storing the Matrix Data: